# Create crossval tables

This notebook is dedicated to create the cv table to all tunings produced during 2020 for tracking purpose.


In [1]:
from core import crossval_table, get_color_fader
from Gaugi import mkdir_p
from copy import copy
from pprint import pprint
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


In [2]:
mkdir_p('output/crossval')
mkdir_p('output/training')

In [3]:
def create_op_dict(op, decoration='reference'):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : decoration+"/"+op+"/pd_ref#0",
              'fa_ref'    : decoration+"/"+op+"/fa_ref#0",
              'sp_ref'    : decoration+"/"+op+"/sp_ref",
              'pd_val'    : decoration+"/"+op+"/pd_val#0",
              'fa_val'    : decoration+"/"+op+"/fa_val#0",
              'sp_val'    : decoration+"/"+op+"/sp_val",
              'pd_op'     : decoration+"/"+op+"/pd_op#0",
              'fa_op'     : decoration+"/"+op+"/fa_op#0",
              'sp_op'     : decoration+"/"+op+"/sp_op",

              # Counts
              'pd_ref_passed'    : decoration+"/"+op+"/pd_ref#1",
              'fa_ref_passed'    : decoration+"/"+op+"/fa_ref#1",
              'pd_ref_total'     : decoration+"/"+op+"/pd_ref#2",
              'fa_ref_total'     : decoration+"/"+op+"/fa_ref#2",
              'pd_val_passed'    : decoration+"/"+op+"/pd_val#1",
              'fa_val_passed'    : decoration+"/"+op+"/fa_val#1",
              'pd_val_total'     : decoration+"/"+op+"/pd_val#2",
              'fa_val_total'     : decoration+"/"+op+"/fa_val#2",
              'pd_op_passed'     : decoration+"/"+op+"/pd_op#1",
              'fa_op_passed'     : decoration+"/"+op+"/fa_op#1",
              'pd_op_total'      : decoration+"/"+op+"/pd_op#2",
              'fa_op_total'      : decoration+"/"+op+"/fa_op#2",
    })
    return d


op_names = ['tight', 'medium', 'loose', 'vloose']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op, "fitting")

In [4]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:


In [5]:
cv_v8  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [6]:
# just for comparison
cv_v8.fill(  '/home/jodafons/public/cern_data/tunings/r1/Zee/v8/*/*/*.npz' , 'v8' )

Reading v8...: 100%|██████████| 4997/4997 [06:34<00:00, 12.66it/s] 

2022-01-25 16:58:07,837 | Py.crossval_table                       INFO Reading file for v8 tag from /home/jodafons/public/cern_data/tunings/r1/Zee/v8/*/*/*.npz
2022-01-25 16:58:07,837 | Py.crossval_table                       INFO There are 4997 files for this task...
2022-01-25 16:58:07,838 | Py.crossval_table                       INFO Filling the table... 
2022-01-25 17:04:42,654 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


In [7]:
cv_v8.to_csv('output/crossval/table_v8.csv')

**NOTE** Use the cell below to rerun all tables from the csv file.

In [8]:
cv_v8.from_csv('output/crossval/table_v8.csv')

### 1.1) Get best inits and sorts:

In [11]:
best_inits_v8 = cv_v8.filter_inits("max_sp_val")
best_sorts_v8 = cv_v8.filter_sorts( best_inits_v8 , 'max_sp_op')
best_inits_v8 = best_inits_v8.loc[best_inits_v8.model_idx==3]

In [12]:
best_inits = pd.concat([
                        best_inits_v8,
            
                       ])
best_sorts = pd.concat([
                        best_sorts_v8,
                       
                       ])

### 1.2) Make the table:

In [14]:
for op in op_names:
    cv_v8.dump_beamer_table( best_inits.loc[best_inits.op_name == op] ,  
                            'tuning_v8_'+op, 
                             title = op+' Tunings (v8)', 
                             tags = ['v8']
                           )

2022-01-25 17:07:10,399 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v8_tight.pdf latex code...
2022-01-25 17:07:11,557 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v8_medium.pdf latex code...
2022-01-25 17:07:12,619 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v8_loose.pdf latex code...
2022-01-25 17:07:13,661 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v8_vloose.pdf latex code...


In [15]:
!mv *.tex output/crossval
!mv *.pdf output/crossval

### 1.3) Plot training curves:

In [16]:
cv_v8.plot_training_curves( best_inits_v8, best_sorts_v8 , 'output/training/v8' )